In [15]:
# import pandas and gurobi
import pandas as pd
from gurobipy import *
import numpy as np


In [20]:
# read in csv
r = pd.read_csv("/Users/zachwayne/Documents/GitHub/15093-project/universe/returns_500.csv")
s = pd.read_csv("/Users/zachwayne/Documents/GitHub/15093-project/universe/sectors_500.csv")
p = pd.read_csv("/Users/zachwayne/Documents/GitHub/15093-project/universe/price_500.csv")
cor = pd.read_csv("/Users/zachwayne/Documents/GitHub/15093-project/universe/correlations_500.csv")
cov = pd.read_csv("/Users/zachwayne/Documents/GitHub/15093-project/universe/covariance_500.csv")

In [21]:
# make model
m = Model("portfolio")
stocks = len(s)
k = 10

In [22]:
# @variable(model, x[1:stocks] >= 0, Int)
x = m.addVars(stocks, vtype=GRB.INTEGER, lb=0, name="x")
# @variable(model, z[1:stocks], Bin);
z = m.addVars(stocks, vtype=GRB.BINARY, name="z")
#@variable(model, 0<=w[1:stocks]<=.15);
w = m.addVars(stocks, vtype=GRB.CONTINUOUS, lb=0, ub=.15, name="w")
#@variable(model,y[1:stocks,1:stocks], Bin);
y = m.addVars(stocks, stocks, vtype=GRB.BINARY, name="y")
ret = m.addVar(vtype=GRB.CONTINUOUS, name="ret")
risk = m.addVar(vtype=GRB.CONTINUOUS, name="risk")

In [23]:
#@constraint(model, sum(x[i]*p[1,i+1] for i=1:stocks) <= 300000);  
m.addConstr(quicksum(x[i]*p.iloc[0,i+1] for i in range(stocks)) <= 300000, "budget")
#@constraint(model, [i=1:stocks], x[i] >= z[i]);
m.addConstrs((x[i] >= z[i] for i in range(stocks)), "x>=z")
#@constraint(model, [i=1:stocks], (x[i]*p[1,i+1]) >= 5000*z[i]);
#m.addConstrs((x[i]*p.iloc[0,i+1] >= 5000*z[i] for i in range(stocks)), "min")
#@constraint(model, [i=1:stocks], w[i] == (x[i]*p[1,i+1])/300000);
m.addConstrs((w[i] == (x[i]*p.iloc[0,i+1])/300000 for i in range(stocks)), "w")
#@constraint(model, [i=1:stocks], 1000000*w[i] >= z[i]);
m.addConstrs((1000000*w[i] >= z[i] for i in range(stocks)), "max")
#@constraint(model, [i=1:stocks], w[i] <= z[i]);
m.addConstrs((w[i] <= z[i] for i in range(stocks)), "w<=z")
#@constraint(model, sum(z[i] for i=1:stocks) == k); 
m.addConstr(quicksum(z[i] for i in range(stocks)) <= k, "k")
#@constraint(model,[j=2:12],sum(z[i]*s[i,j] for i=1:stocks) <= 5);
m.addConstrs((quicksum(z[i]*s.iloc[i,j] for i in range(stocks)) <= 3 for j in range(2,12)), "sector")
#@constraint(model, sum(w[i] for i=1:stocks) == 1);
m.addConstr(quicksum(w[i] for i in range(stocks)) == 1, "sum") 
#@constraint(model, [i=1:stocks, j=1:stocks], y[i,j] >= z[i] + z[j] - 1);
m.addConstrs((x[i] >= z[i] * 5 for i in range(stocks)), "sharesMin")

#for i = 1:stocks
    #for j in 1:stocks
        #if i != j
           # @constraint(model, y[i,j] >= z[i]+z[j]-1)
        #else
           # @constraint(model, y[i,j] == 0)
       # end
   # end
#end

m.addConstrs((y[i,j] >= z[i]+z[j]-1 for i in range(stocks) for j in range(stocks) if i != j), "y")


# for all stocks we purchase, their c[i,j] must be <= .3
#for i=1:stocks
    #for j=1:stocks
           # @constraint(model, y[i,j]*abs(cor[i,j+1]) <= .3)
   # end
#end

# ret = quicksum(w[i]*r.iloc[i,1] for i in range(stocks))
m.addConstr(quicksum(w[i]*r.iloc[i,1] for i in range(stocks)) == ret, "ret")
#risk = quicksum(w[i]*quicksum(w[j]*cov.iloc[i,j+1] for j in range(stocks)) for i in range(stocks))
m.addConstr(quicksum(w[i]*quicksum(w[j]*cov.iloc[i,j+1] for j in range(stocks)) for i in range(stocks)) == risk, "risk")



m.addConstrs((y[i,j]*abs(cor.iloc[i,j+1]) <= .3 for i in range(stocks) for j in range(stocks)), "cor")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

In [24]:
#@objective(model, Min, w'*Matrix(cov[1:stocks, 2:stocks+1])*w - w'*r[1:stocks, 2])
m.setObjective(risk-ret, GRB.MINIMIZE)

In [28]:
# solve model
# Set NonConvex parameter to 2 to solve model.
m.Params.NonConvex = 2
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 462734 rows, 231842 columns and 927338 nonzeros
Model fingerprint: 0xa86512eb
Model has 1 quadratic constraint
Variable types: 482 continuous, 231360 integer (230880 binary)
Coefficient statistics:
  Matrix range     [2e-06, 1e+06]
  QMatrix range    [3e-08, 1e-01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 1e+00]
  RHS range        [3e-01, 3e+05]
Presolved: 465603 rows, 116881 columns, 1200570 nonzeros
Presolved model has 115440 bilinear constraint(s)

Continuing optimization...


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.01179    0   42          -   -0.01179      -     -   10s
     0     0   -0.01179    0   42          -   -0.01179      -     -   1

In [29]:
# print optimal objective
print("Optimal objective: " + str(m.objVal))
  

Optimal objective: inf


In [74]:
# print optimal x
for v in m.getVars():
    if v.x > 0 and v.varName[0] == "x":
        print(p.columns[int(v.varName.replace("x[","").replace("]",""))+1] + ": " + str(v.x))

ABMD: 138.0
AMD: 487.0
CPRT: 7.0
EW: 144.0
ENPH: 259.0
KDP: 515.0
MKTX: 5.0
MTCH: 291.0
NFLX: 85.0
NVDA: 342.0


In [75]:
# find stock in r with highest return
max = 0
maxIndex = 0
for i in range(stocks):
    if r.iloc[i,1] > max:
        max = r.iloc[i,1]
        maxIndex = i

# print stock with highest return
print("Stock with highest return: " + p.columns[maxIndex+1])

# how many kdp to buy till we reach 300000
print("KDP to buy: " + str(300000/p.iloc[0,maxIndex+1]))


ret = 1*r.iloc[maxIndex,1]
w = np.zeros(stocks)
w[maxIndex] = 1
risk = quicksum(w[i]*quicksum(w[j]*cov.iloc[i,j+1] for j in range(stocks)) for i in range(stocks))
print(risk-ret)




Stock with highest return: KDP
KDP to buy: 9818.655660498085
<gurobi.LinExpr: 0.0917925228256703>
